# PATH CONFIG

In [1]:
import os

print(os.getcwd())
if not os.getcwd().endswith("app"):
    os.chdir("../app")
    print(os.getcwd())

# import pandas as pd
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)

%load_ext autoreload
%autoreload 2
# %matplotlib inline

/home/mique/Desktop/Master/TA/TA-Spanish-Esperanto-Translator/notebooks
/home/mique/Desktop/Master/TA/TA-Spanish-Esperanto-Translator/app


In [12]:
import pandas as pd
from dataclasses import dataclass

@dataclass
class Configuration:
    DATA_PATH: str = os.path.join("..", "data")
    RAW_DATA_PATH: str = os.path.join(DATA_PATH, "raw")
    MODELS_PATH: str = os.path.join("..", "models")
    RESULTS_PATH: str = os.path.join("..", "results")

    spanish_data: str = os.path.join(RAW_DATA_PATH, "CCMatrix.eo-es.es")
    esperanto_data: str = os.path.join(RAW_DATA_PATH, "CCMatrix.eo-es.eo")


CONFIG = Configuration()

# EXPERIMENTS

In [ ]:
def read_moses_bitext(source_path, target_path):
    """
    Reads a Moses-formatted parallel corpus from two aligned files.
    
    :param source_path: Path to the source language file (e.g., 'corpus.es')
    :param target_path: Path to the target language file (e.g., 'corpus.eo')
    :return: A list of (source_sentence, target_sentence) tuples
    """
    corpus = []
    
    with open(source_path, 'r', encoding='utf-8') as src_file, \
         open(target_path, 'r', encoding='utf-8') as trg_file:
        
        for src_line, trg_line in zip(src_file, trg_file):
            source_sentence = src_line.strip()
            target_sentence = trg_line.strip()
            
            if source_sentence and target_sentence:
                corpus.append((source_sentence, target_sentence))
    
    return corpus

In [5]:
corpus = read_moses_bitext(CONFIG.spanish_data, CONFIG.esperanto_data)

In [7]:
print(f"{len(corpus):_}")

5_750_168


In [13]:
df_corpus = pd.DataFrame(corpus, columns=["Spanish", "Esperanto"])
df_corpus.head()

,Spanish,Esperanto
0,"Se miran unos a otros, interrogándose con la m...",Ili rigardas unu la alia[n].
1,"""Sí, ¿pero en Brasil no mentirán sobre algo así?","""Jes, sed en Brazilo ili ne mensogos pri io si..."
2,"Has estado en AT5, ellos saben todo ...","Vi estis ĉe AT5, ili scias ĉion ..."
3,Dijeron el Viento y el Bosque: «¡Ninguno de lo...,"* [[1909]]: [[Elżbieta Zawacka]], pola [[kurie..."
4,"Unos a otros se miran,",Ili rigardas unu la alia[n].
